In [ ]:
# Milestone 5

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# Load cleaned dataset
df = pd.read_csv('../data/liver_dataset_cleaned.csv')

# Split features and target
X = df.drop('Target', axis=1)
y = df['Target']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [25]:
# Train default Random Forest
base_model = RandomForestClassifier(random_state=42)
base_model.fit(X_train, y_train)

# Predict and evaluate
y_pred_base = base_model.predict(X_test)

print("📌 Base Model Performance")
print(classification_report(y_test, y_pred_base))


📌 Base Model Performance
              precision    recall  f1-score   support

           0       0.57      0.24      0.34        33
           1       0.75      0.93      0.83        83

    accuracy                           0.73       116
   macro avg       0.66      0.59      0.59       116
weighted avg       0.70      0.73      0.69       116



In [26]:
# Define hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV
grid = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                    param_grid=param_grid,
                    cv=3,
                    n_jobs=-1,
                    verbose=1,
                    scoring='f1')
grid.fit(X_train, y_train)

# Best parameters
print("✅ Best Hyperparameters:", grid.best_params_)


Fitting 3 folds for each of 72 candidates, totalling 216 fits
✅ Best Hyperparameters: {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [28]:
# Predict using best estimator
best_model = grid.best_estimator_
y_pred_tuned = best_model.predict(X_test)

print("📌 Tuned Model Performance")
print(classification_report(y_test, y_pred_tuned))


📌 Tuned Model Performance
              precision    recall  f1-score   support

           0       0.50      0.03      0.06        33
           1       0.72      0.99      0.83        83

    accuracy                           0.72       116
   macro avg       0.61      0.51      0.44       116
weighted avg       0.66      0.72      0.61       116



In [29]:
# Compare metrics manually
def print_comparison(name, y_true, y_pred_base, y_pred_tuned):
    print(f"\n🔍 Comparison for {name}:")
    base = eval(name + '_score')(y_true, y_pred_base)
    tuned = eval(name + '_score')(y_true, y_pred_tuned)
    print(f"Base: {base:.4f} | Tuned: {tuned:.4f}")

for metric in ['accuracy', 'precision', 'recall', 'f1']:
    print_comparison(metric, y_test, y_pred_base, y_pred_tuned)



🔍 Comparison for accuracy:
Base: 0.7328 | Tuned: 0.7155

🔍 Comparison for precision:
Base: 0.7549 | Tuned: 0.7193

🔍 Comparison for recall:
Base: 0.9277 | Tuned: 0.9880

🔍 Comparison for f1:
Base: 0.8324 | Tuned: 0.8325
